# Linear Regression Tutorial
## Complete Guide with Real Data - California Housing Prices

This notebook will teach you linear regression from basics to practical implementation using real-world housing data.

## 1. Import Required Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import scipy.stats as stats
from statsmodels.stats.stattools import durbin_watson
import warnings
warnings.filterwarnings('ignore')

# Set style for better visualizations
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (10, 6)

## 2. Understanding Linear Regression

**Linear Regression** is a statistical method that models the relationship between:
- **Independent variable(s)** (features) - X
- **Dependent variable** (target) - y

The model assumes a linear relationship: $y = \beta_0 + \beta_1X_1 + \beta_2X_2 + ... + \beta_nX_n + \epsilon$

Where:
- $\beta_0$: Intercept
- $\beta_1, \beta_2, ...$: Coefficients
- $\epsilon$: Error term

## 3. Load and Explore the Data

In [ ]:
# Load California Housing dataset
california = fetch_california_housing()
df = pd.DataFrame(california.data, columns=california.feature_names)
df['MedHouseVal'] = california.target * 100000  # Convert to actual dollar amounts

print("📊 Dataset Overview:")
print(f"Shape: {df.shape}")
print(f"Features: {list(df.columns)}")

print("\nFirst 5 rows:")
df.head()

In [ ]:
# Dataset information
print("📈 Dataset Description:")
print(df.describe())

print("\n🔍 Missing Values:")
print(df.isnull().sum())

## 4. Exploratory Data Analysis (EDA)

In [ ]:
# Distribution of target variable
plt.figure(figsize=(15, 5))

plt.subplot(1, 2, 1)
plt.hist(df['MedHouseVal'], bins=50, edgecolor='black', alpha=0.7, color='skyblue')
plt.xlabel('Median House Value ($)')
plt.ylabel('Frequency')
plt.title('Distribution of Median House Values')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
sns.boxplot(y=df['MedHouseVal'], color='lightcoral')
plt.title('Box Plot of Median House Values')
plt.tight_layout()
plt.show()

In [ ]:
# Correlation heatmap
plt.figure(figsize=(10, 8))
correlation_matrix = df.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
            square=True, linewidths=0.5, fmt='.2f')
plt.title('Feature Correlation Heatmap')
plt.tight_layout()
plt.show()

### Observations:
- **MedInc** has the strongest positive correlation with house values (0.69)
- **Latitude** and **AveRooms** also show significant correlations
- **AveRooms** and **AveBedrms** are highly correlated (0.85) - potential multicollinearity

In [ ]:
# Examine relationships with target variable
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

features = ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup']

for i, feature in enumerate(features):
    axes[i].scatter(df[feature], df['MedHouseVal'], alpha=0.5, s=10)
    axes[i].set_xlabel(feature)
    axes[i].set_ylabel('MedHouseVal')
    axes[i].set_title(f'{feature} vs House Value')
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Simple Linear Regression
Let's start with one feature: Median Income

In [ ]:
# Prepare data for simple linear regression
X_simple = df[['MedInc']]
y = df['MedHouseVal']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X_simple, y, test_size=0.2, random_state=42
)

print(f"Training set size: {X_train.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")

In [ ]:
# Create and train the model
simple_model = LinearRegression()
simple_model.fit(X_train, y_train)

# Make predictions
y_pred_simple = simple_model.predict(X_test)

# Model evaluation
mse_simple = mean_squared_error(y_test, y_pred_simple)
rmse_simple = np.sqrt(mse_simple)
mae_simple = mean_absolute_error(y_test, y_pred_simple)
r2_simple = r2_score(y_test, y_pred_simple)

print("🔍 Simple Linear Regression Results:")
print(f"Intercept (β₀): ${simple_model.intercept_:,.2f}")
print(f"Coefficient (β₁): ${simple_model.coef_[0]:,.2f}")
print(f"Mean Squared Error: ${mse_simple:,.2f}")
print(f"Root Mean Squared Error: ${rmse_simple:,.2f}")
print(f"Mean Absolute Error: ${mae_simple:,.2f}")
print(f"R² Score: {r2_simple:.4f}")

### Interpretation:
- **Intercept**: When median income is 0, predicted house value is $45,177 (theoretical baseline)
- **Coefficient**: For each unit increase in median income ($10,000), house value increases by $42,927
- **R²**: 47% of the variance in house values is explained by median income

In [ ]:
# Visualize the simple linear regression
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
# Training data with regression line
plt.scatter(X_train, y_train, alpha=0.5, s=20, label='Training Data', color='blue')
plt.plot(X_train, simple_model.predict(X_train), color='red', 
         linewidth=2, label='Regression Line')
plt.xlabel('Median Income (tens of thousands)')
plt.ylabel('Median House Value ($)')
plt.title('Simple Linear Regression - Training Data')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
# Test data with predictions
plt.scatter(X_test, y_test, alpha=0.5, s=20, label='Actual Test Data', color='green')
plt.scatter(X_test, y_pred_simple, alpha=0.7, s=20, color='red', 
           label='Predicted Values', marker='x')
plt.xlabel('Median Income (tens of thousands)')
plt.ylabel('Median House Value ($)')
plt.title('Predictions vs Actual - Test Data')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Multiple Linear Regression
Now let's use all available features

In [ ]:
# Prepare data for multiple regression
X_multiple = df.drop('MedHouseVal', axis=1)
y = df['MedHouseVal']

# Split the data
X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(
    X_multiple, y, test_size=0.2, random_state=42
)

print("Features used:", list(X_multiple.columns))

In [ ]:
# Create and train multiple regression model
multiple_model = LinearRegression()
multiple_model.fit(X_train_m, y_train_m)

# Make predictions
y_pred_multiple = multiple_model.predict(X_test_m)

# Model evaluation
mse_multiple = mean_squared_error(y_test_m, y_pred_multiple)
rmse_multiple = np.sqrt(mse_multiple)
mae_multiple = mean_absolute_error(y_test_m, y_pred_multiple)
r2_multiple = r2_score(y_test_m, y_pred_multiple)

print("🔍 Multiple Linear Regression Results:")
print(f"Intercept (β₀): ${multiple_model.intercept_:,.2f}")
print("\n📊 Coefficients:")
for feature, coef in zip(X_multiple.columns, multiple_model.coef_):
    print(f"  {feature:12}: ${coef:10,.2f}")
    
print(f"\n📈 Performance Metrics:")
print(f"Mean Squared Error: ${mse_multiple:,.2f}")
print(f"Root Mean Squared Error: ${rmse_multiple:,.2f}")
print(f"Mean Absolute Error: ${mae_multiple:,.2f}")
print(f"R² Score: {r2_multiple:.4f}")

### Interpretation of Coefficients:
- **MedInc**: Strong positive effect - each unit increase adds $40,747 to house value
- **HouseAge**: Positive effect - older houses are generally more valuable
- **AveRooms**: Positive effect - more rooms increase value
- **AveBedrms**: Negative effect - might indicate larger households in less expensive areas
- **Population**: Slight negative effect - more densely populated areas might be less expensive
- **AveOccup**: Negative effect - higher occupancy might indicate lower income areas
- **Latitude**: Strong effect - geographic location significantly impacts price

In [ ]:
# Compare model performance
comparison = pd.DataFrame({
    'Model': ['Simple (MedInc only)', 'Multiple (All Features)'],
    'R² Score': [r2_simple, r2_multiple],
    'RMSE': [rmse_simple, rmse_multiple],
    'MAE': [mae_simple, mae_multiple]
})

print("📊 Model Comparison:")
print(comparison.round(4))

In [ ]:
# Visualize predictions vs actual values
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.scatter(y_test_m, y_pred_multiple, alpha=0.5, color='purple')
plt.plot([y_test_m.min(), y_test_m.max()], [y_test_m.min(), y_test_m.max()], 
         'r--', lw=2, label='Perfect Prediction')
plt.xlabel('Actual Values ($)')
plt.ylabel('Predicted Values ($)')
plt.title('Multiple Regression: Actual vs Predicted')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 2)
residuals = y_test_m - y_pred_multiple
plt.scatter(y_pred_multiple, residuals, alpha=0.5, color='orange')
plt.axhline(y=0, color='r', linestyle='--', label='Zero Residual')
plt.xlabel('Predicted Values ($)')
plt.ylabel('Residuals ($)')
plt.title('Residual Plot')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 3)
plt.hist(residuals, bins=50, edgecolor='black', alpha=0.7, color='lightgreen')
plt.axvline(residuals.mean(), color='red', linestyle='--', label=f'Mean: ${residuals.mean():.2f}')
plt.xlabel('Residuals ($)')
plt.ylabel('Frequency')
plt.title('Distribution of Residuals')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Model Diagnostics and Assumptions

In [ ]:
# Check linear regression assumptions
print("🔍 LINEAR REGRESSION ASSUMPTIONS CHECK:")
print("="*50)

# 1. Linearity assumption
print("\n1. 📈 LINEARITY: Check residual plot above")
print("   - Residuals should be randomly scattered around zero")
print("   - No clear patterns in residual plot")

# 2. Normality of residuals
print("\n2. 📊 NORMALITY OF RESIDUALS:")
print(f"   - Mean of residuals: ${residuals.mean():.2f} (should be near 0)")

# Handle large datasets for Shapiro-Wilk test
if len(residuals) > 5000:
    residuals_sample = residuals.sample(5000, random_state=42)
    stat, p_value = stats.shapiro(residuals_sample)
else:
    stat, p_value = stats.shapiro(residuals)
    
print(f"   - Shapiro-Wilk test p-value: {p_value:.4f}")
print("     (p > 0.05 suggests normality)")

# 3. Homoscedasticity
print("\n3. ⚖️ HOMOSCEDASTICITY:")
print("   - Check residual plot: spread should be constant across predicted values")
print("   - Our plot shows some heteroscedasticity (fan shape)")

# 4. Independence of errors
print("\n4. 🔄 INDEPENDENCE OF ERRORS:")
dw_stat = durbin_watson(residuals)
print(f"   - Durbin-Watson statistic: {dw_stat:.4f}")
print("     (2.0 = no autocorrelation, <1.0 or >3.0 may be problematic)")

In [ ]:
# Q-Q plot for normality check
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
stats.probplot(residuals, dist="norm", plot=plt)
plt.title('Q-Q Plot for Normality Check')

plt.subplot(1, 2, 2)
plt.hist(residuals, bins=50, density=True, alpha=0.7, edgecolor='black', color='lightblue')
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = stats.norm.pdf(x, residuals.mean(), residuals.std())
plt.plot(x, p, 'r', linewidth=2, label='Normal Distribution')
plt.title('Residual Distribution vs Normal')
plt.xlabel('Residuals')
plt.ylabel('Density')
plt.legend()

plt.tight_layout()
plt.show()

## 8. Feature Importance and Interpretation

In [ ]:
# Feature importance based on coefficients
feature_importance = pd.DataFrame({
    'Feature': X_multiple.columns,
    'Coefficient': multiple_model.coef_,
    'Abs_Coefficient': np.abs(multiple_model.coef_)
}).sort_values('Abs_Coefficient', ascending=False)

print("📊 Feature Importance (by coefficient magnitude):")
print(feature_importance.round(2))

In [ ]:
# Visualize feature importance
plt.figure(figsize=(10, 6))
colors = ['green' if x > 0 else 'red' for x in feature_importance['Coefficient']]
bars = plt.barh(feature_importance['Feature'], feature_importance['Coefficient'], color=colors)
plt.xlabel('Coefficient Value ($)')
plt.title('Feature Impact on House Prices')
plt.axvline(x=0, color='black', linestyle='-', alpha=0.3)

# Add value labels on bars
for bar in bars:
    width = bar.get_width()
    plt.text(width, bar.get_y() + bar.get_height()/2, 
             f'${width:,.0f}', 
             ha='left' if width > 0 else 'right', 
             va='center', fontweight='bold')

plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

## 9. Making Predictions - Practical Application

In [ ]:
# Example: Predict house price for a new observation
example_house = {
    'MedInc': [4.5],        # $45,000 median income
    'HouseAge': [25],       # 25 years old
    'AveRooms': [5.2],      # Average 5.2 rooms
    'AveBedrms': [1.1],     # Average 1.1 bedrooms
    'Population': [1200],   # 1200 people in block group
    'AveOccup': [2.8],      # Average 2.8 people per household
    'Latitude': [34.05],    # Southern California
    'Longitude': [-118.24]  # Los Angeles area
}

example_df = pd.DataFrame(example_house)
predicted_price = multiple_model.predict(example_df)[0]

print("🏠 EXAMPLE PREDICTION:")
print("House Characteristics:")
for feature, value in example_house.items():
    print(f"  {feature}: {value[0]}")
print(f"\n💰 Predicted Median House Value: ${predicted_price:,.2f}")

In [ ]:
# Sensitivity analysis: How does changing income affect prediction?
incomes = np.linspace(1, 10, 20)  # From $10,000 to $100,000
sensitivity_df = example_df.copy()

predicted_prices = []
for income in incomes:
    sensitivity_df['MedInc'] = income
    price = multiple_model.predict(sensitivity_df)[0]
    predicted_prices.append(price)

plt.figure(figsize=(10, 6))
plt.plot(incomes * 10000, predicted_prices, 'b-', linewidth=2, marker='o')
plt.xlabel('Median Income ($)')
plt.ylabel('Predicted House Value ($)')
plt.title('Sensitivity Analysis: Income vs House Value')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 10. Key Takeaways and Summary

In [ ]:
print("🎯 KEY TAKEAWAYS:")
print("=" * 60)
print("\n1. 📚 LINEAR REGRESSION BASICS:")
print("   - Models relationship between features and target variable")
print("   - Assumes linear relationship: y = β₀ + β₁X₁ + β₂X₂ + ...")
print("   - Coefficients represent feature importance and direction")

print("\n2. 📊 MODEL PERFORMANCE:")
print(f"   - Simple model (1 feature): R² = {r2_simple:.3f}")
print(f"   - Multiple model (8 features): R² = {r2_multiple:.3f}")
print("   - Adding relevant features improves predictive power")

print("\n3. 🏠 CALIFORNIA HOUSING INSIGHTS:")
print("   - Median income is the strongest predictor of house values")
print("   - Geographic location (latitude) significantly impacts prices")
print("   - House age has a positive but relatively small effect")

print("\n4. ⚠️ MODEL LIMITATIONS:")
print("   - Residuals show heteroscedasticity (uneven variance)")
print("   - Some non-linear relationships may exist")
print("   - Could benefit from feature engineering or polynomial terms")

print("\n5. 🚀 NEXT STEPS TO IMPROVE:")
print("   - Try polynomial features for non-linear relationships")
print("   - Consider regularization (Ridge/Lasso regression)")
print("   - Feature engineering (e.g., room-to-bedroom ratio)")
print("   - Address heteroscedasticity with transformations")

print("\n" + "=" * 60)
print("🎉 Congratulations! You've completed the Linear Regression tutorial!")
print("=" * 60)

In [ ]:
# Save the model and key metrics for future reference
import joblib

# Save the trained model
joblib.dump(multiple_model, 'california_housing_model.pkl')

# Save feature importance
feature_importance.to_csv('feature_importance.csv', index=False)

# Save model performance
performance = pd.DataFrame({
    'Metric': ['R²', 'RMSE', 'MAE'],
    'Simple_Regression': [r2_simple, rmse_simple, mae_simple],
    'Multiple_Regression': [r2_multiple, rmse_multiple, mae_multiple]
})
performance.to_csv('model_performance.csv', index=False)

print("💾 Files saved successfully!")
print("- california_housing_model.pkl (trained model)")
print("- feature_importance.csv (feature coefficients)")
print("- model_performance.csv (model metrics)")